In [2]:
import datetime
import time
from zoneinfo import ZoneInfo

import pandas as pd

from src.dataset import (
    get_clickhouse_client,
    get_timeseries_data,
    store_features_to_center_clickhouse,
    store_features_to_clickhouse,
)
from src.features import calculate_features

MEASUREMENT_NAME = "DongNan"
FIELD_NAME = "空气湿度（%）"  # "空气湿度（%）"、"空气温度（℃）"
ANALYSIS_START_TIME_LOCAL = datetime.datetime(2022, 1, 4, tzinfo=ZoneInfo("Asia/Shanghai"))
ANALYSIS_STOP_TIME_LOCAL = datetime.datetime(2022, 1, 14, tzinfo=ZoneInfo("Asia/Shanghai"))
# 存储--store_features_to_clickhouse
FEATURES_TABLE = "features_caculate_dn"  # "features_caculate"
# 元数据字段
TEMPLE_ID = "045"
DEVICE_ID = "201A"
FREQ = "h"  # h\D\W\M
STATS_CYCLE = "hour"
FEATURE_KEY = "mean"
def main():
    start_time = time.perf_counter()
    # TARGET_DATABASE = "shared"  #  'local'
    # db_client=get_clickhouse_client(target=TARGET_DATABASE)\\

    # raw_df = get_timeseries_data(
    #     measurement_name=MEASUREMENT_NAME,
    #     field_name=FIELD_NAME,
    #     start_time=None,  # ANALYSIS_START_TIME_LOCAL,
    #     stop_time=None,  # ANALYSIS_STOP_TIME_LOCAL,
    # )

    # if raw_df.empty:
    #     print("没有提取到数据，流程结束。")
    #     return

    DATA_FILENAME = "D:\\Projects\\ts_fe\\data\\processed\\preprocessed_data.parquet"

    try:
        raw_df = pd.read_parquet(DATA_FILENAME)
        print("加载成功！")
    except FileNotFoundError:
        print(f"错误：找不到文件 '{DATA_FILENAME}'。")
        exit()

    FEATURE_LIST = ["均值"]
    humidity_features_wide = calculate_features(
        raw_df, field_name=FIELD_NAME, feature_list=FEATURE_LIST, freq=FREQ
    )


    store_features_to_clickhouse(
        df=humidity_features_wide,
        table_name=FEATURES_TABLE,
        field_name=FIELD_NAME,
        device_id=DEVICE_ID,
        temple_id=TEMPLE_ID,
        stats_cycle=STATS_CYCLE,
    )
    end_time = time.perf_counter()
    print(f"\n🎉 ** 总耗时: {end_time - start_time:.2f} 秒 ** 🎉")




In [3]:
main()

加载成功！

正在为字段 '空气湿度（%）' 以 'h' 的频率计算指定的 1 个特征...
✅  calculate_features 执行完毕，耗时: 0.08 秒

开始处理并存入 ClickHouse 的表: 'features_caculate_dn'...
  ► 正在将数据保存到 CSV 文件: features_caculate_dn_20251020_162036.csv ...
  ✔ CSV 文件保存成功: features_caculate_dn_20251020_162036.csv
  ...正在插入 39640 行长格式特征数据...
✅  store_features_to_clickhouse 执行完毕，耗时: 1.04 秒

🎉 ** 总耗时: 1.26 秒 ** 🎉
